In [1]:
from mydbutils import make_connection, do_query_return_all
from pandas import DataFrame
import mysql.connector

In [2]:
conn_warehouse = make_connection(config_file = 'pinnacle_wh.ini')
cursor_warehouse = conn_warehouse.cursor()
conn = make_connection(config_file = 'pinnacle_db.ini')
cursor = conn.cursor()

In [3]:
def make_table(table, sql):
    cursor_warehouse.execute(f"DROP TABLE IF EXISTS {table}")
    cursor_warehouse.execute(sql)

In [4]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
    
    cursor_warehouse.execute(sql)

    columns = cursor_warehouse.description
    column_names = [column_info[0] for column_info in columns]

    df = DataFrame(cursor_warehouse.fetchall())
    df.columns = column_names
    
    return df

# Calendar dimension

In [6]:
sql = ( """
        CREATE TABLE calendar
        (
            calendar_key INT NOT NULL AUTO_INCREMENT,
            full_date DATE,
            day_of_week VARCHAR(9),
            day_of_month INT,
            month INT,
            qtr INT,
            year INT,
            PRIMARY KEY (calendar_key)
        )
        """
      )

make_table('calendar', sql)

In [7]:
sql = ( """
        INSERT INTO pinnacle_wh.calendar(full_date, day_of_week, 
                                            day_of_month, month, 
                                            qtr, year)
            SELECT DISTINCT requiredDate, dayname(requiredDate), 
                            day(requiredDate), month(requiredDate), 
                            quarter(requiredDate), year(requiredDate)
            FROM pinnacle_db.orders
            WHERE status = 'Shipped'
        """
      )

cursor_warehouse.execute(sql)
conn_warehouse.commit()

display_table('calendar', 'full_date')

,calendar_key,full_date,day_of_week,day_of_month,month,qtr,year
0,1,2003-01-13,Monday,13,1,1,2003
1,2,2003-01-18,Saturday,18,1,1,2003
2,3,2003-02-07,Friday,7,2,1,2003
3,4,2003-02-09,Sunday,9,2,1,2003
4,5,2003-02-21,Friday,21,2,1,2003
...,...,...,...,...,...,...,...
244,245,2005-05-13,Friday,13,5,2,2005
245,246,2005-05-14,Saturday,14,5,2,2005
246,247,2005-05-16,Monday,16,5,2,2005
247,248,2005-05-24,Tuesday,24,5,2,2005


# Sales Rep Employee dimension

In [8]:
sql = ( """
        CREATE TABLE salesrepemployee
        (
            employeeNumber INT NOT NULL,
            lastName VARCHAR(50),
            firstName VARCHAR(50),
            email VARCHAR(100),
            managerName VARCHAR(100),
            managerEmail VARCHAR(100),
            PRIMARY KEY(employeeNumber)
        )
        """
      )
make_table('salesrepemployee', sql)

In [9]:
sql = ( """
        INSERT INTO pinnacle_wh.salesrepemployee(employeeNumber, 
                                                   lastName,
                                                   firstName,
                                                   email,
                                                   managerName,
                                                   managerEmail)
            SELECT em1.employeeNumber, em1.lastName, em1.firstName, em1.email, concat(em2.firstName, ' ', em2.lastName) as managerName, em2.email as managerEmail
            FROM pinnacle_db.employees em1
            LEFT JOIN pinnacle_db.employees em2 ON em1.reportsTo = em2.employeeNumber
            WHERE em1.jobTitle = 'Sales Rep'
        """
      )

cursor.execute(sql)
conn.commit()

display_table('salesrepemployee', 'managerName')

,employeeNumber,lastName,firstName,email,managerName,managerEmail
0,1165,Jennings,Leslie,ljennings@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
1,1166,Thompson,Leslie,lthompson@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
2,1188,Firrelli,Julie,jfirrelli@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
3,1216,Patterson,Steve,spatterson@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
4,1286,Tseng,Foon Yue,ftseng@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
5,1323,Vanauf,George,gvanauf@classicmodelcars.com,Anthony Bow,abow@classicmodelcars.com
6,1337,Bondur,Loui,lbondur@classicmodelcars.com,Gerard Bondur,gbondur@classicmodelcars.com
7,1370,Hernandez,Gerard,ghernande@classicmodelcars.com,Gerard Bondur,gbondur@classicmodelcars.com
8,1401,Castillo,Pamela,pcastillo@classicmodelcars.com,Gerard Bondur,gbondur@classicmodelcars.com
9,1501,Bott,Larry,lbott@classicmodelcars.com,Gerard Bondur,gbondur@classicmodelcars.com


# Product Line Dimension

#### Only get products that have been ordered

In [13]:
sql = ( """
        CREATE TABLE productLine
        (
            productLineID INT NOT NULL AUTO_INCREMENT,
            productLineName VARCHAR(100),
            PRIMARY KEY(productLineID)
        )
        """
      )
make_table('productLine', sql)

In [15]:
sql = ( """
        INSERT INTO pinnacle_wh.productLine(productLineName)
            SELECT distinct productLine FROM pinnacle_db.products
        """
      )

cursor.execute(sql)
conn.commit()

display_table('productLine', 'productLineID')

,productLineID,productLineName
0,1,Classic Cars
1,2,Motorcycles
2,3,Planes
3,4,Ships
4,5,Trains
5,6,Trucks and Buses
6,7,Vintage Cars


# Products Dimension

In [5]:
sql = ( """
        CREATE TABLE products
        (
            productCode varchar(15) NOT NULL,
            productName VARCHAR(100),
            productDescription text NOT NULL,
            buyPrice decimal(10,2) NOT NULL,
            PRIMARY KEY(productCode)
        )
        """
      )
make_table('products', sql)

In [6]:
sql = ( """
        INSERT INTO pinnacle_wh.products(productCode,
                                            productName,
                                            productDescription,
                                            buyPrice)
            SELECT productCode, productName, productDescription, buyPrice 
            FROM pinnacle_db.products
            WHERE productCode in (Select productCode from pinnacle_db.orderdetails)
        """
      )

cursor.execute(sql)
conn.commit()

display_table('products', 'productCode')

,productCode,productName,productDescription,buyPrice
0,S10_1678,1969 Harley Davidson Ultimate Chopper,"This replica features working kickstand, front...",48.81
1,S10_1949,1952 Alpine Renault 1300,Turnable front wheels; steering function; deta...,98.58
2,S10_2016,1996 Moto Guzzi 1100i,"Official Moto Guzzi logos and insignias, saddl...",68.99
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,"Model features, official Harley Davidson logos...",91.02
4,S10_4757,1972 Alfa Romeo GTA,Features include: Turnable front wheels; steer...,85.68
...,...,...,...,...
104,S700_3505,The Titanic,"Completed model measures 19 1/2 inches long, 9...",51.09
105,S700_3962,The Queen Mary,Exact replica. Wood and Metal. Many extras inc...,53.63
106,S700_4002,American Airlines: MD-11S,Polished finish. Exact replia with official lo...,36.27
107,S72_1253,Boeing X-32A JSF,"10"" Wingspan with retractable landing gears.Co...",32.77


# Customers Dimension

In [8]:
sql = ( """
        CREATE TABLE customers
        (
            customerNumber INT NOT NULL,
            customerName VARCHAR(100),
            contactLastName VARCHAR(100),
            contactFirstName VARCHAR(100),
            phone VARCHAR(50),
            city VARCHAR(50),
            country VARCHAR(50),
            PRIMARY KEY(customerNumber)
        )
        """
      )
make_table('customers', sql)

In [9]:
sql = ( """
        INSERT INTO pinnacle_wh.customers(customerNumber,
                                            customerName,
                                            contactLastName,
                                            contactFirstName,
                                            phone,
                                            city,
                                            country)
            SELECT customerNumber, customerName, contactLastName, contactFirstName, phone, city, country
            FROM pinnacle_db.customers
            WHERE salesRepEmployeeNumber is not NULL
        """
      )

cursor.execute(sql)
conn.commit()

display_table('customers', 'customerNumber')

,customerNumber,customerName,contactLastName,contactFirstName,phone,city,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,Nantes,France
1,112,Signal Gift Stores,King,Jean,7025551838,Las Vegas,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,Melbourne,Australia
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,Nantes,France
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Stavern,Norway
...,...,...,...,...,...,...,...
95,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,Philadelphia,USA
96,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,Brisbane,USA
97,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,London,UK
98,495,Diecast Collectables,Franco,Valarie,6175552555,Boston,USA


# Sales fact table

In [10]:
sql = ( """
        CREATE TABLE shippedorders
        (
            orderNumber INT NOT NULL,
            calendar_key INT NOT NULL,
            customerNumber INT NOT NULL,
            salesRepEmployeeNumber INT NOT NULL,
            productCode varchar(15) NOT NULL,
            productLineID INT NOT NULL,
            quantityOrdered INT NOT NULL,
            priceEach decimal(10,2) NOT NULL,
            PRIMARY KEY(orderNumber, calendar_key, customerNumber, salesRepEmployeeNumber, productCode, productLineID)
        )
        """
      )

make_table('shippedorders', sql)

In [11]:
sql = ( """
        INSERT INTO pinnacle_wh.shippedorders(orderNumber,
                                                calendar_key,
                                                customerNumber,
                                                salesRepEmployeeNumber,
                                                productCode,
                                                productLineID,
                                                quantityOrdered,
                                                priceEach)
            SELECT o.orderNumber, ca.calendar_key, cus.customerNumber, cus.salesRepEmployeeNumber, 
                od.productCode, pl.productLineID, od.quantityOrdered, od.priceEach FROM pinnacle_db.orders o
                JOIN pinnacle_db.orderdetails od on od.orderNumber = o.orderNumber
                JOIN pinnacle_db.products pro on pro.productCode = od.productCode
                JOIN pinnacle_wh.productline pl on pl.productLineName = pro.productLine
                JOIN pinnacle_db.customers cus on cus.customerNumber = o.customerNumber
                JOIN pinnacle_wh.calendar ca on ca.full_date = o.requiredDate
                WHERE o.status = 'Shipped'
        """
      )

cursor.execute(sql)
conn.commit()

display_table('shippedorders', 'orderNumber')

,orderNumber,calendar_key,customerNumber,salesRepEmployeeNumber,productCode,productLineID,quantityOrdered,priceEach
0,10100,1,363,1216,S18_1749,7,30,136.00
1,10100,1,363,1216,S18_2248,7,50,55.09
2,10100,1,363,1216,S18_4409,7,22,75.46
3,10100,1,363,1216,S24_3969,7,49,35.29
4,10101,2,128,1504,S18_2325,7,25,108.06
...,...,...,...,...,...,...,...,...
2766,10419,249,382,1401,S18_3232,1,35,165.95
2767,10419,249,382,1401,S18_3685,1,43,114.44
2768,10419,249,382,1401,S24_2972,1,15,32.10
2769,10419,249,382,1401,S24_3371,1,55,52.66


# I. Quaterly sales per employee

In [6]:
conn_warehouse = make_connection(config_file = 'pinnacle_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

import pandas as  pd
pd.set_option("display.max_columns", None)
sql = ( """
        SELECT sa.firstName, sa.lastName, sa.managerName, ca.qtr, ca.year, (sh.quantityOrdered*sh.priceEach)
        FROM pinnacle_wh.shippedorders sh
        JOIN pinnacle_wh.salesrepemployee sa ON sa.employeeNumber = sh.salesRepEmployeeNumber
        JOIN pinnacle_wh.calendar ca ON ca.calendar_key = sh.calendar_key
        GROUP BY sa.firstName, sa.lastName, sa.managerName, ca.qtr, ca.year
        ORDER BY sa.firstName, sa.lastName, ca.year, ca.qtr
        """
      )
# print(sql)
rows, count = do_query_return_all(conn_warehouse, sql)
if count > 0:
    df = pd.DataFrame(rows)
    df.columns = ['First Name', 'Last Name', 'Manager Name', 'Quater', 'YearEnd', 'Revenue Made']
print(df.to_string())

    First Name  Last Name       Manager Name  Quater  YearEnd Revenue Made
0         Andy     Fixter  William Patterson       2     2003      3449.26
1         Andy     Fixter  William Patterson       3     2003      2770.78
2         Andy     Fixter  William Patterson       4     2003      4121.95
3         Andy     Fixter  William Patterson       1     2004      2974.43
4         Andy     Fixter  William Patterson       3     2004      3365.10
5         Andy     Fixter  William Patterson       4     2004      4755.60
6         Andy     Fixter  William Patterson       1     2005      4498.55
7        Barry      Jones      Gerard Bondur       1     2003      2701.50
8        Barry      Jones      Gerard Bondur       2     2003      5717.64
9        Barry      Jones      Gerard Bondur       4     2003      1491.38
10       Barry      Jones      Gerard Bondur       1     2004      6051.20
11       Barry      Jones      Gerard Bondur       3     2004      6173.55
12       Barry      Jones

# a) Drill Up - sales per year for employee

In [14]:
sql = ( """
        SELECT sa.firstName, sa.lastName, sa.managerName, ca.year, (sh.quantityOrdered*sh.priceEach)
        FROM pinnacle_wh.shippedorders sh
        JOIN pinnacle_wh.salesrepemployee sa ON sa.employeeNumber = sh.salesRepEmployeeNumber
        JOIN pinnacle_wh.calendar ca ON ca.calendar_key = sh.calendar_key
        GROUP BY sa.firstName, sa.lastName, sa.managerName, ca.year
        ORDER BY sa.firstName, sa.lastName, ca.year
        """
      )
# print(sql)
rows, count = do_query_return_all(conn_warehouse, sql)
if count > 0:
    df = pd.DataFrame(rows)
    df.columns = ['First Name', 'Last Name', 'Manager Name', 'YearEnd', 'Revenue Made']
print(df.to_string())

   First Name  Last Name       Manager Name  YearEnd Revenue Made
0        Andy     Fixter  William Patterson     2003      3449.26
1        Andy     Fixter  William Patterson     2004      2974.43
2        Andy     Fixter  William Patterson     2005      4498.55
3       Barry      Jones      Gerard Bondur     2003      2701.50
4       Barry      Jones      Gerard Bondur     2004      6051.20
5       Barry      Jones      Gerard Bondur     2005      4755.40
6    Foon Yue      Tseng        Anthony Bow     2003      3726.45
7    Foon Yue      Tseng        Anthony Bow     2004      2113.01
8    Foon Yue      Tseng        Anthony Bow     2005      4731.79
9      George     Vanauf        Anthony Bow     2003      2440.50
10     George     Vanauf        Anthony Bow     2004      2832.40
11     George     Vanauf        Anthony Bow     2005      7256.52
12     Gerard  Hernandez      Gerard Bondur     2003      4468.96
13     Gerard  Hernandez      Gerard Bondur     2004      3892.98
14     Ger

# b) Drill down - sales per month for each employee

In [15]:
sql = ( """
        SELECT sa.firstName, sa.lastName, sa.managerName, ca.month, ca.year, (sh.quantityOrdered*sh.priceEach)
        FROM pinnacle_wh.shippedorders sh
        JOIN pinnacle_wh.salesrepemployee sa ON sa.employeeNumber = sh.salesRepEmployeeNumber
        JOIN pinnacle_wh.calendar ca ON ca.calendar_key = sh.calendar_key
        GROUP BY sa.firstName, sa.lastName, sa.managerName, ca.month, ca.year
        ORDER BY sa.firstName, sa.lastName, ca.year, ca.month
        """
      )
# print(sql)
rows, count = do_query_return_all(conn_warehouse, sql)
if count > 0:
    df = pd.DataFrame(rows)
    df.columns = ['First Name', 'Last Name', 'Manager Name', 'Month', 'YearEnd', 'Revenue Made']
print(df.to_string())

    First Name  Last Name       Manager Name  Month  YearEnd Revenue Made
0         Andy     Fixter  William Patterson      5     2003      3449.26
1         Andy     Fixter  William Patterson      7     2003      2770.78
2         Andy     Fixter  William Patterson      9     2003      2636.95
3         Andy     Fixter  William Patterson     10     2003      4121.95
4         Andy     Fixter  William Patterson     11     2003      4903.20
5         Andy     Fixter  William Patterson      2     2004      2974.43
6         Andy     Fixter  William Patterson      7     2004      3365.10
7         Andy     Fixter  William Patterson     12     2004      4755.60
8         Andy     Fixter  William Patterson      2     2005      4498.55
9         Andy     Fixter  William Patterson      3     2005      4680.24
10       Barry      Jones      Gerard Bondur      1     2003      2701.50
11       Barry      Jones      Gerard Bondur      2     2003      5571.80
12       Barry      Jones      Gerard 